In [1]:
from config import Config
import googlemaps

conf = Config()
gClient = googlemaps.Client(key=conf.PLACES_API_KEY)



 API queries_quota: 60 



In [3]:
from time import sleep

I_location = (47.47254804521419, 19.059588492595783)
result = gClient.places_nearby(location=I_location,radius=1000,type='bar')
bars = result['results']
#limited  do while cicle
for _ in range(10):
    #check for paging token
    if 'next_page_token' not in result:
        break
    # credit to https://stackoverflow.com/a/15452661/8659684
    # the docs say, that there is a delay in pages getting public, I guess it is throttling
    sleep(2)
    print('.',end='')
    result = gClient.places_nearby(location=I_location,radius=1000,type='bar',page_token=result['next_page_token'])
    bars += result['results']
print(f' ({len(bars)})')

.. (43)


In [4]:
keys = [k for k in bars[0]]
print(keys,len(bars))

['business_status', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference', 'scope', 'types', 'user_ratings_total', 'vicinity'] 43


In [20]:
import haversine as hs
import pandas as pd

#print(f"{bar['name']} dist: {hs.haversine(I_location,loc,unit=hs.Unit.METERS):4.5}")

df = pd.DataFrame(bars)

[(47.4731179, 19.0517008), (47.4766196, 19.0527181), (47.4778761, 19.0517862), (47.4750225, 19.0489757), (47.4775751, 19.0511132), (47.4776598, 19.0511053), (47.47918159999999, 19.0536033), (47.4782658, 19.0509836), (47.4779625, 19.0507383), (47.4782658, 19.0509836), (47.47736859999999, 19.04946589999999), (47.4788332, 19.0504723), (47.47917349999999, 19.05089570000001), (47.47421800000001, 19.0552939), (47.4732224, 19.0586651), (47.47287660000001, 19.0574087), (47.46953670000001, 19.0638944), (47.4765899, 19.057495), (47.4771838, 19.0537391), (47.46719179999999, 19.06404), (47.4781445, 19.0548387), (47.47756999999999, 19.0534937), (47.4787506, 19.0557218), (47.4777779, 19.0512854), (47.468604, 19.0492154), (47.464729, 19.062781), (47.47842929999999, 19.0513712), (47.47692309999999, 19.0494655), (47.4717675, 19.0718194), (47.4787551, 19.0678338), (47.4786493, 19.0511287), (47.4716139, 19.0469956), (47.4763968, 19.0709533), (47.4795447, 19.0521527), (47.4793487, 19.0515917), (47.4747004

In [32]:
df['location'] = [(g['location']['lat'],g['location']['lng']) for g in df['geometry']]
df['distance'] = [hs.haversine(I_location,g,unit=hs.Unit.METERS) for g in df['location']]
df2 = df.sort_values('distance')
print(df2[['name','distance']])

                                        name    distance
14                            Eufória terasz  102.174089
15                                   Szertár  167.860031
13                              Kamra Campus  372.384304
16                                    Spaten  465.693498
17                               Karman Club  476.181094
0                            Napközi Bisztró  596.224899
18                    Eleven Bistro Budapest  677.485974
19                             Duna Bár Kft.  683.141233
1                                 Pinyó Klub  686.737340
20                                Nemaza Bár  717.420301
21                         Nu Buda Jazz Club  722.259955
22                        Martos Csocsó Klub  748.419101
2                              Black Dog Pub  833.588317
3                                 Fruitisimo  843.777350
4                            Happy Drink Bar  847.475002
23                                 Enter Bar  853.001151
5                              